# Creación y actualiación de database


### Importamos librerías

In [1]:
!pip install apscheduler
!pip install firebase-admin


In [2]:
from apscheduler.schedulers.blocking import BlockingScheduler
import logging
import sqlite3
import firebase_admin
from firebase_admin import credentials, db
import time

### Configurar el logging

In [3]:
count = 1
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

### Definimos la función de inicio de comunicación con FIREBASE y anexar nuevos datos en el dababase

In [4]:
# Definir la función que deseas ejecutar
def mi_tarea():
    
    # URL de Firebase
    firebase_url = 'https://minstationmeteo-default-rtdb.firebaseio.com/datos.json'

    # Inicializa la aplicación de Firebase con las credenciales y la URL de la base de datos
    cred = credentials.Certificate('D:minstationmeteo-firebase-adminsdk-93gzs-fd2a7ac1fa.json')

    if not firebase_admin._apps:
        firebase_admin.initialize_app(cred, {'databaseURL': firebase_url})
    
    # Referencia a la base de datos
    ref = db.reference('/datos')

    # Obtener datos de Firebase
    data = ref.get()
    
    #print(data['Presion Atmosferica Pa'])
    
    data['Presion Atmosferica Pa'] = round((data['Presion Atmosferica Pa']/1000),2)
    #print(data['Presion Atmosferica Pa'])
    #print('')
    print(data)
    
    # Conectar a la base de datos SQLite
    conn = sqlite3.connect('exported_data.db')
    cursor = conn.cursor()

    print("Data actualizada exitosamente")
    
    # Insertar datos en SQLite
    cursor.execute('''
    INSERT INTO mi_table (Unix_Timestamp, Temp2M, RH2M, PreS, rain) VALUES (?, ?, ?, ?, ?)
    ''', (data['timestamp'], data['Temperatura C'], data['Humedad (0-100)% RH'], data['Presion Atmosferica Pa'], data['Rain']))

    # Consultar las últimas 5 filas de la tabla en el mismo orden que la base de datos
    cursor.execute('SELECT * FROM mi_table ORDER BY ROWID DESC LIMIT 5')
    ultimas_filas = cursor.fetchall()

    # Mostrar las últimas 5 filas en el mismo orden que la base de datos
    for fila in reversed(ultimas_filas):
        print(fila)
        
    #GUARDAR EN DATABASE db
    conn.commit()

    # Cerrar la conexión
    conn.close()

### Ejecutamos una vez la función mi_tarea para corrobocar que funciona correctamente

In [5]:
mi_tarea()

{'Humedad (0-100)% RH': 68.6, 'Presion Atmosferica Pa': 75.42, 'Rain': 0, 'Temperatura C': 20.3, 'timestamp': 1737491999.8173292}
Data actualizada exitosamente
(1730728800.0, 18.14, 55.53, 75.46000000000001, 0)
(1730732400.0, 16.91, 60.17, 75.44000000000001, 0)
(1730736000.0, 15.36, 66.94, 75.45, 0)
(1730739600.0, 13.6, 76.68, 75.48, 0)
(1737491999.8173292, 20.3, 68.6, 75.42, 0)


### Creamos un scheduler

#### Es una herramienta que permite programar y gestionar la ejecución de tareas en momentos específicos o a intervalos regulares. Esto es útil para automatizar procesos repetitivos, como enviar correos electrónicos diarios, realizar copias de seguridad, o actualizar bases de datos.

In [6]:
scheduler = BlockingScheduler()


### Bucle principal, para inicializar el scheduler y se ejecute cada 15min

In [7]:
'''
try:
    # Iniciar el scheduler
    scheduler.start()
except (KeyboardInterrupt, SystemExit):
    pass
'''
"""
try:
    # Iniciar el scheduler
    #scheduler.start()
    time.sleep(60)
    mi_tarea()
    count += 1
    logging.info("Datos subidos exitosamente %d", count)
except (KeyboardInterrupt, SystemExit):
    logging.info("Scheduler detenido.")
    pass
"""
while True:
    try:
        # Esperar 60 segundos
        time.sleep(600)
        
        # Ejecutar la tarea
        mi_tarea()
        
        # Incrementar count
        count += 1
        
        # Registrar el mensaje de éxito
        logging.info("Datos subidos exitosamente %d", count)
    except (KeyboardInterrupt, SystemExit):
        logging.info("Scheduler detenido.")
        break  # Salir del bucle si se interrumpe el programa
    except Exception as e:
        logging.error(f"Error al ejecutar la tarea: {e}")
        break  # Salir del bucle si ocurre cualquier otro error

2025-01-21 15:50:36,400 - INFO - Datos subidos exitosamente 2


{'Humedad (0-100)% RH': 73.3, 'Presion Atmosferica Pa': 75.42, 'Rain': 0, 'Temperatura C': 19.5, 'timestamp': 1737492595.7028453}
Data actualizada exitosamente
(1730732400.0, 16.91, 60.17, 75.44000000000001, 0)
(1730736000.0, 15.36, 66.94, 75.45, 0)
(1730739600.0, 13.6, 76.68, 75.48, 0)
(1737491999.8173292, 20.3, 68.6, 75.42, 0)
(1737492595.7028453, 19.5, 73.3, 75.42, 0)


2025-01-21 16:00:36,950 - INFO - Datos subidos exitosamente 3


{'Humedad (0-100)% RH': 75.5, 'Presion Atmosferica Pa': 75.44, 'Rain': 0, 'Temperatura C': 19.2, 'timestamp': 1737493231.218341}
Data actualizada exitosamente
(1730736000.0, 15.36, 66.94, 75.45, 0)
(1730739600.0, 13.6, 76.68, 75.48, 0)
(1737491999.8173292, 20.3, 68.6, 75.42, 0)
(1737492595.7028453, 19.5, 73.3, 75.42, 0)
(1737493231.218341, 19.2, 75.5, 75.44, 0)


2025-01-21 16:10:37,555 - INFO - Datos subidos exitosamente 4


...

2025-01-21 22:54:45,734 - INFO - Datos subidos exitosamente 40


{'Humedad (0-100)% RH': 100.0, 'Presion Atmosferica Pa': 75.76, 'Rain': 0, 'Temperatura C': 12.1, 'timestamp': 1737518052.983871}
Data actualizada exitosamente
(1737515664.8077586, 12.0, 100.0, 75.76, 0)
(1737516266.0466824, 11.9, 100.0, 75.77, 0)
(1737516861.6132507, 12.4, 100.0, 75.78, 0)
(1737517457.436435, 11.9, 100.0, 75.77, 0)
(1737518052.983871, 12.1, 100.0, 75.76, 0)


2025-01-21 23:04:46,707 - INFO - Datos subidos exitosamente 41


{'Humedad (0-100)% RH': 100.0, 'Presion Atmosferica Pa': 75.76, 'Rain': 0, 'Temperatura C': 12.2, 'timestamp': 1737518648.9908118}
Data actualizada exitosamente
(1737516266.0466824, 11.9, 100.0, 75.77, 0)
(1737516861.6132507, 12.4, 100.0, 75.78, 0)
(1737517457.436435, 11.9, 100.0, 75.77, 0)
(1737518052.983871, 12.1, 100.0, 75.76, 0)
(1737518648.9908118, 12.2, 100.0, 75.76, 0)


2025-01-22 08:31:16,605 - WARNING - Retrying (Retry(total=2, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001DD8A170FA0>: Failed to establish a new connection: [WinError 10051] Se ha intentado una operación de socket en una red no accesible')': /token
2025-01-22 08:31:16,648 - WARNING - Retrying (Retry(total=1, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001DD8A1C5220>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /token
2025-01-22 08:31:16,649 - WARNING - Retrying (Retry(total=0, connect=None, read=None, redirect=None, status=None)) after connection broken by 'NewConnectionError('<urllib3.connection.HTTPSConnection object at 0x000001DD8A1CAA30>: Failed to establish a new connection: [Errno 11001] getaddrinfo failed')': /token
2025-01-22 08:31

### Programamos para que se ejecute este scheduler cada 15min y anexe nuevos datos

In [ ]:
#scheduler.add_job(mi_tarea, 'interval', minutes=1)
